In [1]:
import torch



In [2]:
# # A. mask / cut out

# import torch
# import glob
# import cv2
# import os
# import imgaug.augmenters as iaa

# # sample_token as name
# image_list = []
# ori_img_name_list = []
# damage_label = 'A'

# image_path = 'image_gray_200_original'
# images = sorted(glob.glob(image_path + '/*'))[0:900]


# for i in range(len(images)):

#         image = cv2.imread(images[i])
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         image_list.append(image)
#         ori_img_name = os.path.basename(images[i])
#         ori_img_name_list.append(ori_img_name)


# # print(len(image_list))
    
# seq = iaa.Sequential([
#     iaa.Cutout(nb_iterations=(4, 6), size=0.2, squared=False, random_state = 1 )
#     , ])

# images_aug_list = seq.augment_images(image_list)
# print(len(images_aug_list))

# for index in range(len(images_aug_list)):
#     # s = '%05d' % (index+1) # name format 00001
#     s = ori_img_name_list[index]
#     cv2.imwrite(f'testdataset/damaged_image/{damage_label}_{s}', images_aug_list[index])

# print('successful!')


32
successful!


In [5]:
# A. for later regionla mse

import numpy as np
import cv2
from matplotlib import pyplot as plt
import glob
import os

def create_mask(image_shape, block_size, num_blocks):
    mask = np.ones(image_shape, dtype=np.float32) 
    block_size_px = (int(image_shape[0] * block_size), int(image_shape[1] * block_size))
    # np.random.seed(5)

    for _ in range(num_blocks):
        top_left = np.random.randint(0, image_shape[0] - block_size_px[0] + 1), np.random.randint(0, image_shape[1] - block_size_px[1] + 1)
        bottom_right = top_left[0] + block_size_px[0], top_left[1] + block_size_px[1]
        mask[top_left[0]:bottom_right[0], top_left[1]:bottom_right[1]] = 0
        
    return mask


image_list = []
ori_img_name_list = []
image_path = 'image_gray_200_original'
images = sorted(glob.glob(image_path + '/*'))[0:1]
damage_label = 'A'

for i in range(len(images)):
    image = cv2.imread(images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])

    block_size = 0.2  # 方块大小为原图大小的百分比
    num_blocks = 5  # 方块数量
    mask = create_mask(image.shape[:2], block_size, num_blocks)
    damaged_img1 = image * mask
    # cv2.imwrite(f'image_gray_200_damaged/{ori_img_name}', damaged_img1)
    cv2.imwrite('damage/1.png',damaged_img1)

# # 显示损坏的图像，保留 mask 的效果
# plt.imshow(damaged_img1, cmap='gray')
# plt.show()


In [6]:
# B. motionblur

# loading library
import random
import glob
import cv2
import numpy as np
from pathlib import Path


def motion_blur(img):
    # Specify the kernel size.
    # The greater the size, the more the motion.
    kernel_size = random.randint(40, 70)

    # # Create the vertical kernel.
    kernel_v = np.zeros((kernel_size, kernel_size))

    # Create a copy of the same for creating the horizontal kernel.
    kernel_h = np.copy(kernel_v)

    # Fill the middle row with ones.
    # kernel_v[:, int((kernel_size - 1) / 2)] = np.ones(kernel_size)
    kernel_h[int((kernel_size - 1) / 2), :] = np.ones(kernel_size)

    # Normalize.
    # kernel_v /= kernel_size
    kernel_h /= kernel_size

    # # Apply the vertical kernel.
    # vertical_mb = cv2.filter2D(img, -1, kernel_v)

    # Apply the horizontal kernel.
    horizonal_mb = cv2.filter2D(img, -1, kernel_h)

    return horizonal_mb

    # Save the outputs.
    # cv2.imwrite('car_vertical.jpg', vertical_mb)
    
damage_label = 'B'
images_aug_list = []
ori_img_name_list = []

image_path = 'image_gray_200_original'
images = sorted(glob.glob(image_path + '/*'))[0:1]

for i in range(len(images)):
    image = cv2.imread(images[i])
    image = motion_blur(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])

    # cv2.imwrite(f'image_gray_200_damaged/{ori_img_name}', image)
    cv2.imwrite('damage/2.png',image)


print('successful!')


successful!


In [8]:
# C. sunflare
import torch
import os
import random
import glob
import cv2
import albumentations as A

damage_label = 'C'

image_path = 'image_gray_200_original'
images = sorted(glob.glob(image_path + '/*'))[0:1]

print(len(images))
# fixed src center
# 一个list里面加的都一样！不采纳！
# flare_images= am.add_sun_flare(image, flare_center=(x,y), angle=-math.pi/4)
# hp.visualize(flare_images, column=3)

for index in range(len(images)):
    img = cv2.imread(images[index])

    transform = A.Compose(
         [A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0.5, p=1, src_radius =200)],
    )
    random.seed()
    transformed = transform(image=img)
    img = transformed['image']
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[index])
    # cv2.imwrite(f'image_gray_200_damaged/{ori_img_name}', image)
    cv2.imwrite('damage/3.png',image)


print('successful!')


1
successful!


In [9]:
# D. foggu

import glob
import cv2
import imgaug.augmenters as iaa

# sample_token as name
image_list = []
ori_img_name_list = []

damage_label = 'D'


image_path = 'image_gray_200_original'
images = sorted(glob.glob(image_path + '/*'))[0:1]


# for i in range(len(images)):
#     image = cv2.imread(images[i])
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     ori_img_name = os.path.basename(images[i])
#     # cv2.imwrite(f'../../Dataset/lidar_img_16200/damaged_image_gray/D_fog/{ori_img_name}', image)
    
    
    
# for i in range(len(images)):

#         image = cv2.imread(images[i])
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         image_list.append(image)
#         ori_img_name = os.path.basename(images[i])
#         ori_img_name_list.append(ori_img_name)


# seq = iaa.Sequential([
#     iaa.imgcorruptlike.Fog(severity=3),] )

# images_aug_list = seq.augment_images(image_list)

# for index in range(len(images_aug_list)):
#     s = ori_img_name_list[index]
#     cv2.imwrite(f'../../Dataset/lidar_img_16200/damaged_image_gray/D_fog/{s}', images_aug_list[index])


for i in range(len(images)):
    image = cv2.imread(images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_list.append(image)


seq = iaa.Sequential([
    iaa.imgcorruptlike.Fog(severity=5),] )

images_aug_list = seq.augment_images(image_list)

for index in range(len(images_aug_list)):
    ori_img_name = os.path.basename(images[index])
    # cv2.imwrite( f'image_gray_200_damaged/{ori_img_name}', images_aug_list[index])
    cv2.imwrite('damage/4.png',images_aug_list[index])


In [11]:
# E. overexposure

import cv2
import numpy as np
import random
import glob


def overexposure(img):
    # 将bgr转化为hsv
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img = img.astype(np.float64)
    # 获取v通道（颜色亮度通道），并做渐变性的增强
    Exp1 = random.randint(75, 90)
    Exp2 = random.randint(55, 65)
    Exp3 = random.randint(15, 25)

    img[:, :, 2] = np.where(img[:, :, 2] > 40, img[:, :, 2] + Exp1, img[:, :, 2])
    img[:, :, 2] = np.where(img[:, :, 2] > 140, img[:, :, 2] + Exp2, img[:, :, 2])
    img[:, :, 2] = np.where(img[:, :, 2] > 180, img[:, :, 2] + Exp3, img[:, :, 2])

    # 令大于255的像素值等于255（防止溢出）
    img = np.where(img > 255, 255, img)
    img = img.astype(np.uint8)
    res = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
    return res

images_aug_list = []
ori_img_name_list = []
damage_label = 'E'

image_path = 'image_gray_200_original'
images = sorted(glob.glob(image_path + '/*'))[0:1]

print(len(images))
for i in range(len(images)):
    image = cv2.imread(images[i])
    image = overexposure(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])
    # cv2.imwrite(f'../../Dataset/lidar_img_16200/damaged_image_gray/E_overexposure/{ori_img_name}', image)
    # cv2.imwrite(f'image_gray_200_damaged/{ori_img_name}', image)
    cv2.imwrite('damage/5.png',image)


1


In [14]:
# F. underexposure
import cv2
import numpy as np
import random
import glob

def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

image_path = 'image_gray_200_original'
images = sorted(glob.glob(image_path + '/*'))[0:1]
damage_label = 'F'


for i in range(len(images)):
    image = cv2.imread(images[i])
    gamma = random.uniform(0.15, 0.3)
    image = adjust_gamma(image, gamma)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])
    # cv2.imwrite(f'image_gray_200_damaged/{ori_img_name}', image)
    cv2.imwrite('damage/6.png',image)


In [15]:
# G condensed water
import torch
import glob
import cv2
import os
import imgaug.augmenters as iaa

image_list=[]
image_path = 'image_gray_200_original'
images = sorted(glob.glob(image_path + '/*'))[0:1]
damage_label = 'G'

print(len(images))
for i in range(len(images)):
    image = cv2.imread(images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_list.append(image)


seq = iaa.Sequential([
    iaa.imgcorruptlike.Spatter(severity=2),] )

images_aug_list = seq.augment_images(image_list)

for index in range(len(images_aug_list)):
    ori_img_name = os.path.basename(images[index])
    # cv2.imwrite( f'image_gray_200_damaged/{ori_img_name}', images_aug_list[index])
    cv2.imwrite('damage/7.png',images_aug_list[index])


1


In [17]:
# H powder cluster

image_path = 'image_gray_200_original'
image_list=[]
images = sorted(glob.glob(image_path + '/*'))[0:1]
damage_label = 'H'

for i in range(len(images)):
    image = cv2.imread(images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_list.append(image)


seq = iaa.Sequential([
    iaa.imgcorruptlike.Spatter(severity=5),] )

images_aug_list = seq.augment_images(image_list)

for index in range(len(images_aug_list)):
    ori_img_name = os.path.basename(images[index])
    
    # cv2.imwrite( f'image_gray_200_damaged/{ori_img_name}', images_aug_list[index])
    cv2.imwrite('damage/8.png',images_aug_list[index])





In [64]:
# I regionalPixelShift
import random
import glob
import cv2
import numpy as np

CAM_FRONT_path = '../../Dataset/lidar_img_16200/original_image_gray'
images = sorted(glob.glob(CAM_FRONT_path + '/*'))[0:1]

def pixel_shift(img):
    out = img.copy()
    out_mid = img.copy()
    out_mid2 = img.copy()
    shift1 = random.randint(5, 10)
    width1 = random.randint(12, 20)
    x1 = random.randint(0, 1600 - 3 * width1)

    out[:, x1 - 3 * width1:x1, :] = out_mid[:, x1:x1 + 3 * width1, :]
    out[:, x1:x1 + 3 * width1, :] = out_mid2[:, x1 - 3 * width1:x1, :]
    # out[x1 - shift1:x1 - shift1 + width1, :, :] = out[x1:x1 + width1, :, :]
    return out


for i in range(5):
    image = cv2.imread(images[i])
    image = pixel_shift(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])
    # cv2.imwrite(f'../../Dataset/lidar_img_16200/damaged_image_gray/I_regionalPixelShift/{ori_img_name}', image)
    cv2.imwrite('damage/9.png',image)

    

In [63]:
# test the geerated gray image
# Import necessary libraries
import torch
from PIL import Image
import torchvision.transforms as transforms
img=Image.open('../../Dataset/lidar_img_16200/damaged_image_gray/F_underexposure/8e9c6424a0e642dc97d75baf6d4c6d6f.png')

print(type(img))

transform = transforms.Compose([
    transforms.PILToTensor()
])

img_tensor = transform(img)
print(type(img_tensor))
print(img_tensor.shape)


<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'torch.Tensor'>
torch.Size([1, 256, 256])


In [1]:
# create only the masked dataset 
import torch
import glob
import cv2
import os
import imgaug.augmenters as iaa

# sample_token as name
image_list = []
ori_img_name_list = []

CAM_FRONT_path = '../../Dataset/lidar_img_16200/original_image_gray'
images = sorted(glob.glob(CAM_FRONT_path + '/*'))

for i in range(len(images)):

        image = cv2.imread(images[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image_list.append(image)
        ori_img_name = os.path.basename(images[i])
        ori_img_name_list.append(ori_img_name)


# print(len(image_list))
    
seq = iaa.Sequential([
    iaa.Cutout(nb_iterations=(16, 24), size=0.2, squared=False, seed =1 )
    , ])

images_aug_list = seq.augment_images(image_list)

for index in range(len(images_aug_list)):
    # s = '%05d' % (index+1) # name format 00001
    s = ori_img_name_list[index]
    cv2.imwrite(f'../../Dataset/lidar_img_16200/maksed_image_gray/{s}', images_aug_list[index])


In [13]:

images_aug_list = []
ori_img_name_list = []


CAM_FRONT_path = '../../Dataset/lidar_img_16200/original_image_gray'
# images = sorted(glob.glob(CAM_FRONT_path + '/*'))[1800:3600]
images = sorted(glob.glob(CAM_FRONT_path + '/*'))[14624:14656]
print(len(images))
for i in range(len(images)):
    image = cv2.imread(images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])
    cv2.imwrite(f'../../Dataset/lidar_img_16200/damaged_image_gray_all/for_porcessing/{ori_img_name}', image)

32


In [4]:
import torch
import cv2
import glob
import os

image_list = []
ori_img_name_list = []
image_path = 'testdataset/damaged_image'
images = sorted(glob.glob(image_path + '/*'))


for i in range(256,288):
    image = cv2.imread(images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])


    cv2.imwrite(f'testdataset/damaged_image/{ori_img_name}', image)